<a href="https://colab.research.google.com/github/edwardb1203/HACKNC2022/blob/main/HACKNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report
from sklearn import metrics


In [ ]:
import gradio as gr

In [ ]:
url = 'https://raw.githubusercontent.com/edwardb1203/HACKNC2022/main/Crop_recommendation_dataset.csv'
df=pd.read_csv(url)
df.head()

In [ ]:
# Splitting our features and targets
features = df[['N', 'P','K','temperature', 'humidity', 'ph', 'rainfall']]
target = df['label']

In [ ]:
# Splitting into test and train data
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size = 0.25,random_state = 1)

In [ ]:
# Training our model... we are using extreme gradient boost
import xgboost as xgb
XB = xgb.XGBClassifier()
XB.fit(Xtrain,Ytrain)

# Take a look at the accuracy
x = metrics.accuracy_score(Ytest, predicted_values)
print("XGBoost's Accuracy is: ", x)


In [ ]:
# Saving our trained model
with open("XGBoost_example.pkl", "wb") as f:
  pickle.dump(XB, f)

In [ ]:
# Loading trained model
with open("XGBoost_example.pkl", "rb") as g:
  XGB = pickle.load(g)

In [ ]:
# This is where all of our descriptions are stored
grow_guides = {	
'rice':'When to plant: Plant in late spring\nWater level: Rice requires an abundance of water\nSun level:  Plant in full sun, or 6-8 hours of sunlight',
'maize': 'When to plant: Plant 2-3 weeks after your area’s last frost\nWater level: 2 inches of water per week\nSun level:  Plant in full sun, or 6-8 hours of sunlight',
'chickpea': 'When to plant: Plant 1-2 weeks before your area’s last frost\nWater Level: One inch of water per week\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'kidneybeans': 'When to plant: Plant in the spring, after your area’s last frost\nWater Level:  Keep the plant moist,  but avoid overwatering\nSun Level: Plant in full sun, or 6-8 hours of sunlight',
'pigeonpeas':'When to plant: Plant in late spring after soil temperatures have risen\nWater levels: grow in well draining soil\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'mothbeans':'When to plant: Plant throughout Summer\nWater levels: Very little irrigation required\nSun levels: Plant in full sun, or 6-8 hours of sunlight',
'mungbean': 'When to plant: Plant in spring when soil temperatures exceed 65 degrees Fahrenheit\nWater level: Thrives in moderately dry conditions\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'blackgram':'When to plant: Black gram is generally grown in the rainy summer season\nWater level: Plant in well draining soil\nSun level: Full sun, however an excess of heat requires an increase in the frequency of irrigation',
'lentil': 'When to plant: Plant 2-3 weeks before the last frost date of your area\nWater level: Keep evenly moist in loose, well drained soil\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'pomegranate': 'When to plant: Plant in spring, or fall in warmer areas\nWater level:  Water pomegranates deeply during the summer\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'banana': 'When to plant: Plant in the spring, banana trees generally have a fast growth rate\nWater level: Water regularly to ensure soil stays moist, bananas prefer a humid environment\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'mango': 'When to plant: Plant in the spring when the weather is still mild\nWater level: Water whenever the top 2 inches of the soil dry out\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'grapes': 'When to plant: Plant in early spring after last hard freeze has occured\nWater Level: Before planting, soak roots in water for 2 hours, water at time of planting\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'watermelon': 'When to plant: Start seeds indoors 2-3 weeks before you last frost date, transplant to garden after 2 weeks\nWater level: 1-2 inches of water per week\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'muskmelon': 'When to plant: 3-4 weeks after the last average frost date of your area\nWater Level: 1-2 inches of water per week\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'apple': 'When to plant: Plant in early spring\nWater level: Water young trees regularly to ensure that root systems become established\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'orange': 'When to plant: Plant in early spring, or when frost is no longer a threat\nWater level: Citrus plants need well draining soil to prevent over watering and root rot\nSun level: Plant in full sun, or 6-8 hours of sunlight, warmth helps the citrus sweeten!',
'papaya': 'When to plant: Papayas grow best in temperatures between 70 and 90 degrees F\nWater Level: As the plant grows, water it every 3-4 days\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'coconut': 'When to plant: Coconuts prefer high humidity and temperatures between 90 - 100 degrees F\nWater Level: 1 inch per week\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'cotton': 'When to plant: Cotton germinates best in temperatures between 75-80 degrees F\nWater Level: Keep cotton moist, but not wet to avoid root rot\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'jute': 'When to plant: Plant jute seeds between early spring, and late spring\nWater Level: Water jute once a week, it requires consistent rainfall as well\nSun level: Plant in full sun, or 6-8 hours of sunlight',
'coffee': 'When to plant: Late spring is the best time to plant Coffee\nWater Level: Keep the soil moist, but not soaking well, with good drainage\nSun level: Coffee plants prefer partial, indirect sunlight'
}

# This is the function that takes in our model parameters
# It creates a dataframe with our parameters because our model predict call is expecting a dataframe
# Then we store the predicted crop and use it to get the correct image url and description from our dict
def make_prediction(N, P, K, temperature, humidity, ph, rainfall):
    predictors = pd.DataFrame([[N,P,K,temperature,humidity,ph,rainfall]], columns=['N','P','K','temperature','humidity','ph','rainfall'])
    crop = XGB.predict(predictors)[0]
    url = "https://raw.githubusercontent.com/edwardb1203/HACKNC2022/main/assets/" + crop + ".jpg"
    description = grow_guides[crop]
    return crop, url, description

# This is all of the input we need to collect
N = gr.Number(label = "Enter Nitrogen content of soil: ")
P = gr.Number(label= "Enter Phosphorous content of soil: ")
K = gr.Number(label = "Enter Potassium content of soil: ")
temperature = gr.Number(label = "Enter approximate local temperature: ")
humidity = gr.Number(label = "Enter approximate local humidity %: ")
ph = gr.Number(label = "Enter ph of soil: ")
rainfall = gr.Number(label = "Enter approximate local rainfall for the season in mm: ")

# This is what we map our output to 
output0 = gr.Textbox(label="Optimal Crop:")
output1 = gr.Image(label="Crop Image")
output2 = gr.Textbox(label="Crop Info:")

# Storing the input in an array to pass to function
input_arr=[N, P, K, temperature, humidity, ph, rainfall]

# Gradio app launcher
app = gr.Interface(fn = make_prediction, inputs=input_arr, outputs=[output0, output1, output2])
app.launch(share=True)
